# WORK-IN-PROGRESS
Last updated 2020-04-16

To-do
- ~~See why abstract is missing in example `rec_info` towards the end of the notebook~~ <-- doesn't happen anymore
- ~~Check quality of recommendations~~ <-- seems better with scibert
- Write out brief explanations
- Clean up cells a little
- ~~Try using scibert~~
- ~~Play with preprocessing~~ <-- got rid of all special characters

In [1]:
# Set the environment path to find Recommenders
import sys
sys.path.append("../../")

# Import functions
from reco_utils.dataset.covid_utils import *
from reco_utils.recommender.tfidf.tfidf_utils import *

# Print version
print("System version: {}".format(sys.version))

System version: 3.7.5 (default, Oct 31 2019, 15:18:51) [MSC v.1916 64 bit (AMD64)]


## Parameters

In [2]:
# Parameters for the COVID-19 dataset in Azure Open Datasets
# https://azure.microsoft.com/en-us/services/open-datasets/catalog/covid-19-open-research/
azure_storage_account_name='azureopendatastorage'
azure_storage_sas_token='sv=2019-02-02&ss=bfqt&srt=sco&sp=rlcup&se=2025-04-14T00:21:16Z&st=2020-04-13T16:21:16Z&spr=https&sig=JgwLYbdGruHxRYTpr5dxfJqobKbhGap8WUtKFadcivQ%3D'
container_name='covid19temp'
metadata_filename='metadata.csv'

## Load in the data

Download metadata.csv (takes about 1.5 min)

In [3]:
# Get metadata
blob_service = get_blob_service(azure_storage_account_name, azure_storage_sas_token, container_name)
metadata = download_metadata(blob_service, container_name, metadata_filename)

Only keep metadata for articles that are under public domain (cc0)

In [4]:
# Extract metadata on public domain articles only
metadata_public = extract_public_domain(metadata)

# Clean dataframe
metadata_public = clean_dataframe(metadata_public)
metadata_public.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_pdf_parse,has_pmc_xml_parse,full_text_file,url
106,ej795nks,a3c3b7c38ad32e1042d78aae2027ca491e9f2197,PMC,Understanding the Spatial Clustering of Severe...,10.1289/ehp.7117,PMC1247620,15531441.0,cc0,We applied cartographic and geostatistical met...,2004-07-27,"Lai, P.C.; Wong, C.M.; Hedley, A.J.; Lo, S.V.;...",Environ Health Perspect,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
108,9mzs5dl4,c1c6a98c21304f3788b20870b34afd8a115fa38c,PMC,The Application of the Haddon Matrix to Public...,10.1289/ehp.7491,PMC1257548,15866764.0,cc0,State and local health departments continue to...,2005-02-02,"Barnett, Daniel J.; Balicer, Ran D.; Blodgett,...",Environ Health Perspect,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
152,u7lz3spe,c56ffdaf1cfbae5a6ed0abea495eaf7fa1cbc031,PMC,Cynomolgus Macaque as an Animal Model for Seve...,10.1371/journal.pmed.0030149,PMC1435788,16605302.0,cc0,BACKGROUND: The emergence of severe acute resp...,2006-04-18,"Lawler, James V; Endy, Timothy P; Hensley, Lis...",PLoS Med,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
160,na7z92i8,f38f3b112e4b702b60ba56be806d418bbb2b83c3,PMC,Immune Protection of Nonhuman Primates against...,10.1371/journal.pmed.0030177,PMC1459482,16683867.0,cc0,BACKGROUND: Ebola virus causes a hemorrhagic f...,2006-05-16,"Sullivan, Nancy J; Geisbert, Thomas W; Geisber...",PLoS Med,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
212,j35w1vsw,5dc0b8b662824323881c3a1ae3a1bae2a821484d,PMC,SARS: Systematic Review of Treatment Effects,10.1371/journal.pmed.0030343,PMC1564166,16968120.0,cc0,BACKGROUND: The SARS outbreak of 2002–2003 pre...,2006-09-12,"Stockman, Lauren J; Bellamy, Richard; Garner, ...",PLoS Med,NaN,NaN,True,True,comm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...


Retrieve the full text available for all the public domain articles (takes about 2.5 min)

In [5]:
# Extract text from all public domain articles
all_text = get_public_domain_text(metadata_public, blob_service, container_name)
all_text.head()

../..\reco_utils\dataset\covid_utils.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text'][row] = retrieve_text(df.iloc[row], blob_service, container_name)
C:\Users\Administrator\.conda\envs\rec_repo\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,index,cord_uid,doi,title,publish_time,authors,journal,url,abstract,full_text
0,0,ej795nks,10.1289/ehp.7117,Understanding the Spatial Clustering of Severe...,2004-07-27,"Lai, P.C.; Wong, C.M.; Hedley, A.J.; Lo, S.V.;...",Environ Health Perspect,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,We applied cartographic and geostatistical met...,Since the emergence and rapid spread of the e...
1,1,9mzs5dl4,10.1289/ehp.7491,The Application of the Haddon Matrix to Public...,2005-02-02,"Barnett, Daniel J.; Balicer, Ran D.; Blodgett,...",Environ Health Perspect,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,State and local health departments continue to...,"sudden fever and dry cough, along with chills..."
2,2,u7lz3spe,10.1371/journal.pmed.0030149,Cynomolgus Macaque as an Animal Model for Seve...,2006-04-18,"Lawler, James V; Endy, Timothy P; Hensley, Lis...",PLoS Med,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,BACKGROUND: The emergence of severe acute resp...,The emergence of severe acute respiratory syn...
3,3,na7z92i8,10.1371/journal.pmed.0030177,Immune Protection of Nonhuman Primates against...,2006-05-16,"Sullivan, Nancy J; Geisbert, Thomas W; Geisber...",PLoS Med,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,BACKGROUND: Ebola virus causes a hemorrhagic f...,Background Ebola virus causes a hemorrhagic f...
4,4,j35w1vsw,10.1371/journal.pmed.0030343,SARS: Systematic Review of Treatment Effects,2006-09-12,"Stockman, Lauren J; Bellamy, Richard; Garner, ...",PLoS Med,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,BACKGROUND: The SARS outbreak of 2002–2003 pre...,The SARS outbreak of 2002-2003 presented clin...


In [6]:
len(all_text)

247

In [7]:
all_text['full_text'][0]

' Since the emergence and rapid spread of the etiologic agent of severe acute respiratory syndrome (SARS)-SARS coronavirus (SARS-CoV)-in late 2002 and during the first 6 months of 2003, great progress has been made in understanding the biology, pathogenesis, and epidemiology of both the disease and the virus (SARS-CoV). Much remains to be done, however, including the development of effective therapeutic interventions and diagnostic tools with high sensitivity and specificity soon after the onset of clinical symptoms. The evaluation of key epidemiologic parameters and the impact of different public health interventions in the various settings that experienced minor or major epidemics is also needed (Affonso et al. 2004; Cui et al. 2003; Lau et al. 2004; Leung et al., in press) . In terms of outbreak control on the population level, many questions about "superspreading events" (SSEs) remain to be investigated. Such an SSE was responsible for > 300 cases (out of a total of 1,755) in the A

## Preprocessing

In [8]:
# ------------- PREPROCESSING ---------------------
# Assign columns to clean and combine
cols_to_clean = ['abstract','full_text']
df_clean = clean_dataframe_for_rec(all_text, cols_to_clean, for_BERT=True)
df_clean.head()

../..\reco_utils\recommender\tfidf\tfidf_utils.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][i] = clean_text(df[col][i], for_BERT)
../..\reco_utils\recommender\tfidf\tfidf_utils.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = df['title'][i].encode('ascii','ignore')
../..\reco_utils\recommender\tfidf\tfidf_utils.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = df['title'][i].decode(

,index,cord_uid,doi,title,publish_time,authors,journal,url,abstract,full_text,cleaned_text
0,0,ej795nks,10.1289/ehp.7117,Understanding the Spatial Clustering of Severe...,2004-07-27,"Lai, P.C.; Wong, C.M.; Hedley, A.J.; Lo, S.V.;...",Environ Health Perspect,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,We applied cartographic and geostatistical met...,Since the emergence and rapid spread of the e...,We applied cartographic and geostatistical met...
1,1,9mzs5dl4,10.1289/ehp.7491,The Application of the Haddon Matrix to Public...,2005-02-02,"Barnett, Daniel J.; Balicer, Ran D.; Blodgett,...",Environ Health Perspect,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,State and local health departments continue to...,sudden fever and dry cough along with chills ...,State and local health departments continue to...
2,2,u7lz3spe,10.1371/journal.pmed.0030149,Cynomolgus Macaque as an Animal Model for Seve...,2006-04-18,"Lawler, James V; Endy, Timothy P; Hensley, Lis...",PLoS Med,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,BACKGROUND The emergence of severe acute respi...,The emergence of severe acute respiratory syn...,BACKGROUND The emergence of severe acute respi...
3,3,na7z92i8,10.1371/journal.pmed.0030177,Immune Protection of Nonhuman Primates against...,2006-05-16,"Sullivan, Nancy J; Geisbert, Thomas W; Geisber...",PLoS Med,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,BACKGROUND Ebola virus causes a hemorrhagic fe...,Background Ebola virus causes a hemorrhagic f...,BACKGROUND Ebola virus causes a hemorrhagic fe...
4,4,j35w1vsw,10.1371/journal.pmed.0030343,SARS: Systematic Review of Treatment Effects,2006-09-12,"Stockman, Lauren J; Bellamy, Richard; Garner, ...",PLoS Med,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,BACKGROUND The SARS outbreak of 20022003 prese...,The SARS outbreak of 20022003 presented clini...,BACKGROUND The SARS outbreak of 20022003 prese...


In [9]:
# Preview
df_clean['cleaned_text'][0]

'We applied cartographic and geostatistical methods in analyzing the patterns of disease spread during the 2003 severe acute respiratory syndrome SARS outbreak in Hong Kong using geographic information system GIS technology We analyzed an integrated database that contained clinical and personal details on all 1755 patients confirmed to have SARS from 15 February to 22 June 2003 Elementary mapping of disease occurrences in space and time simultaneously revealed the geographic extent of spread throughout the territory Statistical surfaces created by the kernel method confirmed that SARS cases were highly clustered and identified distinct disease hot spots Contextual analysis of mean and standard deviation of different density classes indicated that the period from day 1 18 February through day 16 6 March was the prodrome of the epidemic whereas days 86 15 May to 106 4 June marked the declining phase of the outbreak Originanddestination plots showed the directional bias and radius of spre

## Recommender

Create recommendation matrix (takes about 30 sec)

In [10]:
# ------------- RECOMMEND ---------------------
# TF-IDF with tokenization_method of 'nltk', 'bert', 'scibert', or 'none'
results = recommend_with_tfidf(df_clean,
                               text_col='cleaned_text',
                               id_col='cord_uid',
                               title_col='title',
                               tokenization_method='scibert')

In [11]:
# Organize results as an easy-to-read table
rec_table = organize_results_as_tabular(df_clean,
                                        results,
                                        id_col='cord_uid',
                                        k=5)
rec_table.head()

,cord_uid,title,rec_rank,rec_score,rec_cord_uid,rec_title
0,ej795nks,Understanding the Spatial Clustering of Severe...,1,0.141480,u7lz3spe,Cynomolgus Macaque as an Animal Model for Seve...
1,ej795nks,Understanding the Spatial Clustering of Severe...,2,0.116403,j35w1vsw,SARS: Systematic Review of Treatment Effects
2,ej795nks,Understanding the Spatial Clustering of Severe...,3,0.100148,nt60lv2k,A Network Integration Approach to Predict Cons...
3,ej795nks,Understanding the Spatial Clustering of Severe...,4,0.076347,vp9d9vmp,Genome Wide Identification of SARS-CoV Suscept...
4,ej795nks,Understanding the Spatial Clustering of Severe...,5,0.074854,05d1mhkq,A Porcine Epidemic Diarrhea Virus Outbreak in ...


### Show single recommendation

In [12]:
# For example
rec_id = rec_table['rec_cord_uid'][0]

In [13]:
# Get full information for any particular recommendation
rec_info = get_full_info(all_text, rec_id)

"Cynomolgus Macaque as an Animal Model for Severe Acute Respiratory Syndrome" (2006) by Lawler, James V; Endy, Timothy P; Hensley, Lisa E; Garrison, Aura; Fritz, Elizabeth A; Lesar, May; Baric, Ralph S; Kulesh, David A; Norwood, David A; Wasieloski, Leonard P; Ulrich, Melanie P; Slezak, Tom R; Vitalis, Elizabeth; Huggins, John W; Jahrling, Peter B; Paragas, Jason.
Available at https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1435788/
